In [1]:
#https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

In [2]:
from keras.preprocessing.image import ImageDataGenerator,array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

Using TensorFlow backend.


In [3]:
# dimensions of our images.
#img_width, img_height = 640, 480
img_width, img_height = 150, 150


train_data_dir = '../dataset/split_data/train/'
validation_data_dir = '../dataset/split_data/validation/'
nb_train_samples = 13447
nb_validation_samples = 4487
epochs = 1
batch_size = 25

In [4]:
def create_model():
    # if K.image_data_format() == 'channels_first':
    #     input_shape = (3, img_width, img_height)
    # else:
    input_shape = (img_width, img_height, 3)

    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10))
    model.add(Activation('sigmoid'))
    
    return model

In [5]:
model = create_model()

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')

Found 13447 images belonging to 10 classes.
Found 4487 images belonging to 10 classes.
Epoch 1/1
  8/537 [..............................] - ETA: 9:41 - loss: 2.4814 - acc: 0.1050

KeyboardInterrupt: 

In [ ]:
model = create_model()
model.load_weights('luisa.h5')
img = load_img("../dataset/split_data/validation/c0/img_87085.jpg", target_size=(img_width, img_height))
x = img_to_array(img)
x = x.reshape((1,) + x.shape)
print(x.shape)
classes = model.predict_proba(x, verbose=1)
print(classes)